<h1>Data preprocessing!</h1>

In [1]:
# Usual import
from pprint import pprint
import numpy as np
import pandas as pd
from tqdm import tqdm
import string

# Gensim
import gensim
import gensim.corpora as corpora

#For language detection
from langdetect import detect

# spaCy for tokenization, stop word removal and  lemmatization
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

<h2>Load Data</h2>

In [159]:
data = pd.read_csv("C:/Users/Iro Sfoungari/Desktop/THESIS DATA/CORD-19/metadata.csv")
data.head(2)

C:\Users\Iro Sfoungari\anacondaDisaster\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [161]:
len(data)

381817

In [162]:
data["abstract"]=data["abstract"].astype(str)

We just care about the text in abstracts now, so we remove all the NaN abstracts

In [163]:
data2 = data[data.abstract != 'nan']


In [164]:
len(data2)

273771

In [167]:
#data = pd.read_csv("C:/Users/Iro Sfoungari/Desktop/data_with_lang.csv")


Language detection. There where some symbols in the text, and detect(x) couldn't work! so I had to add an exception 

In [168]:
def language_detection(x):
    try:
        lang = detect(x)
    except:
        lang = 'Other'
    return lang


In [ ]:
tqdm.pandas()
data2["Lang"] = data2["abstract"].progress_apply(language_detection)

C:\Users\Iro Sfoungari\anacondaDisaster\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


  0%|                                                                                       | 0/273771 [00:00<?, ?it/s]

  0%|                                                                             | 4/273771 [00:00<2:12:21, 34.47it/s]

  0%|                                                                            | 10/273771 [00:00<1:55:33, 39.49it/s]

  0%|                                                                            | 16/273771 [00:00<1:46:27, 42.86it/s]

  0%|                                                                            | 21/273771 [00:00<1:43:42, 43.99it/s]

  0%|                                                                            | 26/273771 [00:00<1:43:17, 44.17it/s]

  0%|                           

store it in a csv locally

In [11]:
#csv_data = data2.to_csv(r'C:/Users/Iro Sfoungari/Desktop/data_with_lang_certaincols.csv', index = False)

drop all rows that contain fr, spanish, ge or other langyages

In [12]:
data3=data2[data2['Lang'].str.contains("en")]

So we have 269223 abstracts to process!

In [13]:
len(data3)

956

In [113]:
data3.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,processed_description,Lang,processed_description2
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,...,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,objective retrospective chart review describes...,en,"[objective, retrospective, chart, review, desc..."
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,...,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,inflammatory diseases respiratory tract common...,en,"[inflammatory, diseases, respiratory, tract, c..."
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,...,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,surfactant protein d sp d participates innate ...,en,"[surfactant, protein, d, sp, d, participates, ..."
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,...,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,endothelin-1 et-1 21 amino acid peptide divers...,en,"[endothelin-1, et-1, 21, amino, acid, peptide,..."
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,...,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,respiratory syncytial virus rsv pneumonia viru...,en,"[respiratory, syncytial, virus, rsv, pneumonia..."


<h2> Data preprocessing / Cleaning </h2>
Tokening the Data With spaCy

Start by mporting the English models we need from spaCy, as well as Python’s string module, which contains a  list of all punctuation marks that we can use in string.punctuation.


In [147]:
# Create our list of punctuation marks
punctuations = string.punctuation
stopwords = list(STOP_WORDS)
nlp = spacy.load('en_core_web_sm')
parser = English()

In [148]:
# Creating our tokenizer function

def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)
    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    # return preprocessed list of tokens
    #mytokens = " ".join([i for i in mytokens])
    return mytokens

In [149]:
data3["processed_description2"] = data3["abstract"].progress_apply(spacy_tokenizer)




  0%|                                                                                          | 0/956 [00:00<?, ?it/s]

  2%|█▉                                                                              | 23/956 [00:00<00:04, 229.59it/s]

  5%|████▏                                                                           | 50/956 [00:00<00:03, 230.78it/s]

  8%|██████                                                                          | 73/956 [00:00<00:03, 221.85it/s]

 11%|████████▌                                                                      | 103/956 [00:00<00:03, 236.16it/s]

 13%|██████████▋                                                                    | 129/956 [00:00<00:03, 240.52it/s]

 16%|████████████▋                                                                  | 154/956 [00:00<00:03, 242.35it/s]

 19%|███████████████▏                                                               | 184/956 [00:00<00:03, 253.34it/s]

 23%|██████████████████       

Gensim is a Python library for topic modelling, document indexing and similarity retrieval with large corpora.

In [150]:
data  = data3.processed_description2.values.tolist()


In [151]:
print(data[0])

['objective', 'retrospective', 'chart', 'review', 'describes', 'epidemiology', 'clinical', 'features', '40', 'patients', 'culture', 'proven', 'mycoplasma', 'pneumoniae', 'infections', 'king', 'abdulaziz', 'university', 'hospital', 'jeddah', 'saudi', 'arabia', 'methods', 'patients', 'positive', 'm.', 'pneumoniae', 'cultures', 'respiratory', 'specimens', 'january', '1997', 'december', '1998', 'identified', 'microbiology', 'records', 'charts', 'patients', 'reviewed', 'results', '40', 'patients', 'identified', '33', '82.5', 'required', 'admission', 'infections', '92.5', 'community', 'acquired', 'infection', 'affected', 'age', 'groups', 'common', 'infants', '32.5', 'pre', 'school', 'children', '22.5', 'occurred', 'year', 'round', 'common', 'fall', '35', 'spring', '30', 'quarters', 'patients', '77.5', 'comorbidities', 'isolates', '60', 'associated', 'pneumonia', '14', '35', 'upper', 'respiratory', 'tract', 'infections', '2', '5', 'bronchiolitis', 'cough', '82.5', 'fever', '75', 'malaise', '5

<h2> Tokenize words and Clean-up text</h2>


Gensim’s simple_preprocess() is great for this. Additionally I have set deacc=True to remove the punctuations.

stopword removal

In [152]:
# Define functions for stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]




In [153]:
data_words = list(sent_to_words(data))
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [154]:
print(data_words[0])

['objective', 'retrospective', 'chart', 'review', 'describes', 'epidemiology', 'clinical', 'features', 'patients', 'culture', 'proven', 'mycoplasma', 'pneumoniae', 'infections', 'king', 'abdulaziz', 'university', 'hospital', 'jeddah', 'saudi', 'arabia', 'methods', 'patients', 'positive', 'pneumoniae', 'cultures', 'respiratory', 'specimens', 'january', 'december', 'identified', 'microbiology', 'records', 'charts', 'patients', 'reviewed', 'results', 'patients', 'identified', 'required', 'admission', 'infections', 'community', 'acquired', 'infection', 'affected', 'age', 'groups', 'common', 'infants', 'pre', 'school', 'children', 'occurred', 'year', 'round', 'common', 'fall', 'spring', 'quarters', 'patients', 'comorbidities', 'isolates', 'associated', 'pneumonia', 'upper', 'respiratory', 'tract', 'infections', 'bronchiolitis', 'cough', 'fever', 'malaise', 'common', 'symptoms', 'crepitations', 'wheezes', 'common', 'signs', 'patients', 'pneumonia', 'crepitations', 'bronchial', 'breathing', '

<h2> Bigrams</h2>
Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold. The higher the values of these param, the harder it is for words to be combined to bigrams.

In [155]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])


['objective', 'retrospective', 'chart', 'review', 'describes', 'epidemiology', 'clinical', 'features', 'patients', 'culture', 'proven', 'mycoplasma', 'pneumoniae', 'infections', 'king', 'abdulaziz', 'university', 'hospital', 'jeddah', 'saudi', 'arabia', 'methods', 'patients', 'positive', 'pneumoniae', 'cultures', 'respiratory', 'specimens', 'january', 'december', 'identified', 'microbiology', 'records', 'charts', 'patients', 'reviewed', 'results', 'patients', 'identified', 'required', 'admission', 'infections', 'community_acquired', 'infection', 'affected', 'age', 'groups', 'common', 'infants', 'pre', 'school', 'children', 'occurred', 'year', 'round', 'common', 'fall', 'spring', 'quarters', 'patients', 'comorbidities', 'isolates', 'associated', 'pneumonia', 'upper', 'respiratory_tract', 'infections', 'bronchiolitis', 'cough', 'fever', 'malaise', 'common', 'symptoms', 'crepitations', 'wheezes', 'common', 'signs', 'patients', 'pneumonia', 'crepitations', 'bronchial', 'breathing', 'patien

In [156]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


In [157]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words)


In [158]:
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [130]:
data_words_trigrams = make_trigrams(data_words_bigrams)


In [131]:
print(data_words_trigrams[0])

['objective', 'retrospective', 'chart', 'review', 'describes', 'epidemiology', 'clinical', 'features', 'patients', 'culture', 'proven', 'mycoplasma', 'pneumoniae', 'infections', 'king', 'abdulaziz', 'university', 'hospital', 'jeddah', 'saudi', 'arabia', 'methods', 'patients', 'positive', 'pneumoniae', 'cultures', 'respiratory', 'specimens', 'january', 'december', 'identified', 'microbiology', 'records', 'charts', 'patients', 'reviewed', 'results', 'patients', 'identified', 'required', 'admission', 'infections', 'community_acquired', 'infection', 'affected', 'age', 'groups', 'common', 'infants', 'pre', 'school', 'children', 'occurred', 'year', 'round', 'common', 'fall', 'spring', 'quarters', 'patients', 'comorbidities', 'isolates', 'associated', 'pneumonia', 'upper', 'respiratory_tract', 'infections', 'bronchiolitis', 'cough', 'fever', 'malaise', 'common', 'symptoms', 'crepitations', 'wheezes', 'common', 'signs', 'patients', 'pneumonia', 'crepitations', 'bronchial', 'breathing', 'patien

In [132]:

# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [144]:
#spacy lemmatization

def lemmatization(texts):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc])
    return texts_out


In [145]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams)


In [146]:
print(data_lemmatized[0])

['objective', 'retrospective', 'chart', 'review', 'describe', 'epidemiology', 'clinical', 'feature', 'patient', 'culture', 'prove', 'mycoplasma', 'pneumoniae', 'infection', 'king', 'abdulaziz', 'university', 'hospital', 'jeddah', 'saudi', 'arabia', 'method', 'patient', 'positive', 'pneumoniae', 'culture', 'respiratory', 'specimen', 'january', 'december', 'identify', 'microbiology', 'record', 'chart', 'patient', 'review', 'result', 'patient', 'identify', 'required', 'admission', 'infection', 'community_acquired', 'infection', 'affect', 'age', 'group', 'common', 'infant', 'pre', 'school', 'child', 'occur', 'year', 'round', 'common', 'fall', 'spring', 'quarter', 'patient', 'comorbiditie', 'isolate', 'associate', 'pneumonia', 'upper', 'respiratory_tract', 'infection', 'bronchiolitis', 'cough', 'fever', 'malaise', 'common', 'symptom', 'crepitation', 'wheeze', 'common', 'sign', 'patient', 'pneumonia', 'crepitation', 'bronchial', 'breathing', 'patient', 'likely', 'non', 'patient', 'present', 